In [1]:
import numpy as np
import csv

from modules.cas_TS import RecSys


import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
items_loc = 'csv_files/items.csv'
data_loc = 'users_data/users_data.csv'
parameters_loc = 'parameters/parameters.pickle'

In [3]:
K = 6 #number of recommendations per user
n = 25 #total number of days/rounds the simulations run for
alpha = 0 #diversity factor
number_of_users = 100 #number of users per day
from_scratch = 0 #indicator specifying wether the training should start from scratch or continue from the latest trained parameteres



In [4]:
items = pd.read_csv(items_loc)
titles = list(items['IDENTIFIER'])
ctr = {t:0.01*np.random.randint(1,K) for t in titles}
click_probabilities = np.array(list(ctr.values()))
L = items.shape[0] #number of mindfulness items

In [5]:
regrett = []
repeatt = np.zeros([10000, n])
days = 10000*[[]]


Rec = RecSys(items, K, L, alpha)

In [6]:
for day in range(1, n+1):
    
    if day>1:
        data = pd.read_csv(data_loc)
    else:
        data = {}

    Rec.train(parameters_loc, data, from_scratch)
    for user in range(10000):
        repeatt[user][day-1] = Rec.repeat[user]
        
    header = ['selected']+[r'choice{}'.format(i+1) for i in range(K)] +['user_id']  
    with open(data_loc, 'w') as file:
        writer = csv.writer(file)
        writer.writerow(header)


    regs = 0    
    user_ids = np.arange((day-1)*(number_of_users/50), number_of_users+(day-1)*(number_of_users/50), dtype=int)
    for user_id in user_ids:
        df1 = pd.read_csv(data_loc)
        recommendations = Rec.predict(parameters_loc, user_id)
        Rec.make_recommendation_and_save_new_data(click_probabilities, recommendations, data_loc, user_id)
        df2 = pd.read_csv(data_loc)
        regs = regs+Rec.regret
        if len(df1)<len(df2) and day<n:
            days[user_id] = days[user_id]+[day]
        
        
    regrett.append(regs)
        


In [8]:
#ploting regret curve
q = 0
p = n
regret = np.array([regrett[day] for day in range(n)])
ff = np.array([sum(regret[0:t])/number_of_users for t in range(1,n+1)])
plt.plot(ff[q:p]/np.arange(q+1,p+1))
plt.xlabel(r'Round, $t$')
plt.ylabel(r'Per-step regret, $\frac{R_t}{t}$')

In [18]:
#plotting the repeatition curve for the user with maximum number of repeatition
q = []
for i in range(10000):
    if len(days[i])>0:
        q.append(i)

f = np.zeros([10000])
for user_id in range(10000):
    f[user_id] = max(repeatt[user_id])
    
user_id = q[f[q].argsort()[-100:][::-1][0]]


plt.plot(repeatt[user_id][days[user_id]])
plt.ylim(-0.1,K+0.1)
plt.xlabel(r'Round, $t$')
plt.ylabel(r'Repetition')